# Többváltozós analízis mérnöki alkalmazásai, 6. gyakorlat: Robotok és grafika

In [1]:
from graphics import * #mindent behúzok grafikából, nem kell utána gr.-to írni

In [2]:
win =GraphWin("Első ablakunk", 600, 400)
win.getMouse()
win.close()
# így lehet ablakot csinálni, ha pontosan tudjuk, meddig kell ->animációhoz pl nem jó

In [3]:
win =GraphWin("Alakzatok", 600, 400)
win.setBackground("black")

pont=Point(200,200)#pont objektum: x,y koordináták megadásával
pont.setFill(color_rgb(69,14,88)) #színt így is állíthatjuk

pont.draw(win)

kor=Circle(pont,3)
kor.setFill("white") #outline és fill között
kor.draw(win)

kor2=Circle(pont,100)
kor2.setOutline("white") #előtte: mit rontottam el? (alapméretezett: fekete)
kor2.draw(win)

egyenes=Line(pont, Point(300,200))#így belül is lehet, néha így kényelmesebb
egyenes.setOutline("white")
egyenes.draw(win)

win.getMouse()
win.close()

## 1.feladat: Legyen 3D!

Nem csak 2 dimenzidós dolgokat szeretünk megjelíteni - érdemes kipróbálni 3D-ben is! Írjunk egy függvényt, ami 3D-s vektorokból Cavalier axonometria használatával 2D-s Point objektumokat ad vissza, majd egy (600,400)-as fekete canvason megjeleníti az egységvektorokat! (Segítség: eltolást opcionálisan tegyetek bele, mert később nagyon kellhet!)

## Készítsünk animációt! (először 2D)

In [8]:
from graphics import *
import numpy as np


 
win =GraphWin("Fourier", 400, 400, autoflush=False)#animáció készítésénél: ld. Zelle pdf-je!
win.setBackground("black")

t=0 #legyen egy változónk az időhöz: a cos, sin függvényebe ezt fgojuk beletenni
radius = 100; #kör sugara
center=Point(200,200) #ez legyen a kp


while win.checkMouse()==None: #azért nem az egyszerűbb megoldást használjuk,mert nem tudjuk, meddig néznénk a kört
    if(t>np.pi*2):
        t=0
    
    win.delete("all")#letöröljük az előző képkockát
    x=center.x#kör megrajzolása
    y=center.y
    
    point_1=Point(x,y)
    circ = Circle(point_1,radius)
    circ.setOutline("white")
    circ.draw(win)
    
    x+=radius*np.cos(t)#forgó pont koordinátái
    y+=radius*-np.sin(t)
    
    
        
    point_2=Point(x,y)#ez lesz a forgó pont
        
    radius_show= Line(point_1, point_2)#sugár megrajzolása
    radius_show.setFill("white")
    radius_show.draw(win)
            
    t+=0.02
    update(20) #magyarázat a pdf-ben.
     
win.close() ##ezt külön feltöltöm, el kell játszani vele. Növeljék a forgási sebességet, a sugarat, tegyék át másik pontba...

## Forgassunk 3D-ben!

Az előbb megírt függvényeteket használva forgassatok meg egy z irányú pálcát a z tengely körül (vmilyen sugárral, hogy láttszon is).

# Robotika

![robotka1](https://images.adsttc.com/media/images/525d/4735/e8e4/4e67/bf00/09ab/large_jpg/AU130517_medium.jpg?1381844767)


A robotok manipulátorai - a leglátványosabb részeik - bonyolult, térbeli mozgásokat végző szerkezetek. A mai órán a robotokat és a koordinátatranszformációkat fogjuk kicsit közelebb hozni egymáshoz. Beszélünk a robotok működéséről, és belekóstolunk abba is, hogy mi a helyzetek a robotikában mint iparágban Magyarországon.

## A robotok működése

Az, amit általában a képeken láttok, csak a robotok egy része. A látványos karon kívül kell hozzájuk még egy kontroller és egy speciális tápegység is, mert nagyteljesítményű gépekről van szó - 1 kW körül kezdődnek a motorjaik. Ahhoz, hogy a működésüket megértsük, fontos fejben tartani, hogy mi az egyetlen funkciójuk: pontos pályán mozgatják végig a robot szermszámát (TCP-t), hogy azzal munkát végezzünk.


![TCP](https://www.heidenhain.us/wp-content/uploads/Bild2-web_v2-1.jpg)


A robotok karjait ízületekre tagolhatjuk ugyanúgy, mint az élőlényekét. Minden ízületben egy motortengely (axis) található. Minden ízület - általában - egy szabaságfokot ad a robotszerszámnak. Nem szeretünk fölöslegesen dolgozni, ezért mindig épp anni DoF-t veszünk, amennyi kell.

- Mire jó 3 DoF? [Kompano - üvegház](https://www.priva.com/discover-priva/news-and-stories/priva-kompano-deleaf-line)
- Mire jó 4 DoF? Pakoló robi
- Mire jó 6 DoF? Kb minden
- Mire jó 7 DoF? Benyúlni, kikerülni. Kollab.

Láttuk ezt a sok nyúlkáló dolgot, most nézzünk utána, hogy mi van bennük. Az ipari robotikában - a nagy teljesítmények miatt - nem találunk léptetőmotort, mindegyik ipari robot AC (néha DC) szervomotorokról megy. A szervo annyit tesz, hogy (egy enkóderen) vissza van csatolva a robot pozíciója.

![Motorok](http://cn.rccn.com.cn/upload/image/20171225/20171225083440_37570.jpg)

Találjuk meg a motorokat! Nem mind a tengely mellett van (Direct drive...)

![belul](https://i.ytimg.com/vi/HXJOnWBbcwM/maxresdefault.jpg)

Átbeszéltük, hogy mi van egy robotkarban: csak motorok, a hozzájuk tartozó hajtás, meg vas. Innentől arra a kérdésre fogunk választ keresni, hogy hogyan tudják a számítógépek ezeket az otrombaságokat mozgatni - feltételezzük, hogy a motorok szabályozása tökéletes, azokkal nem kell törődnünk (akit ilyesmi érdekel, később felveheti a Stépán-féle robotok mechanikáját).

## Kinematikai láncok

A robotokat karokra bonthatjuk, amiket csuklók (joint) tagolnak, ezek mentén tudnak elmozdulni: rotációs vagy transzlációs mozgást végezhetnek (rotational/prismatic joint) - mi csak az előbbivel foglalkozunk most (példa utóbbira - kompano). Az ízületek sora egy láncot alkot.

### Direkt (forward) kinematika

Csuklók elmozdulásából/motorok állásából mondjuk meg, hogy merre jár és hogy háll a TCP. A dolgunk az, vegyük a TCP koordinátarendszerének origóját, visszahaladjunk a kinematikai láncon a végéig, majd a világkoordinátarendszerben megadjuk TCP koordinátáit. Egy "egyszerű" koordinátatranszformáció az egész!

### Inverz kinematika

Na ez az igazi! Mondjuk meg, hogy milyen motorpozíciók kellenek az adott TCP álláshoz. Ezt biztos nem csináljuk ma! 

## Direkt kinematikai feladat megoldása

Azért ez sem mindig könnyű - mi a különdség a két robot között? (nyílt lánc/zárt lánc)


![robotka](https://i.pinimg.com/originals/f0/14/1b/f0141b7f5e7d5801b7e2acceb646c6dd.png)

![gifke](https://j.gifs.com/wVmz8R.gif)

Az előbbivel (nyílt kinematikai lánc) lényegesen könnyebb a dolgunk, ezekkel fogunk dolgozni.

### Nyilt kinematikai láncó robotok direkt kinematikája

A feladatunk tehát az, hogy egy nyílt láncú robot TCP-jének koordinátáit megmondjuk csak a motorpozíciókból.

Koordináta-transzformációkat fogunk használni: kiindulunk a robot végéből, és addig lépegetünk visszafele, amíg az origóba nem érünk.

![kepecskem](https://realitybytesdotblog.files.wordpress.com/2017/06/planarbot.png?w=788)

Koordinátatranszformációkról tanultatok! Mondjatok nekem egyet, amivel megadhatom $B$ pontból az $E$-ben megadott koordinátákat!

Nincs ilyen, mert nem lineáris a leképezés. Ezért vezetjük be a ([link](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&cad=rja&uact=8&ved=2ahUKEwjnl_KZqfrgAhVGM-wKHafxAu8QFjAAegQIAhAC&url=https%3A%2F%2Fwww.tankonyvtar.hu%2Fhu%2Ftartalom%2Ftamop425%2F0008_mester%2FMester_Robotika.pdf&usg=AOvVaw1Cku32cYg9vUB-5kNMFMVM) az összes forrással) 4x4-es formalizmust, egy eltolásvektorral kiegészítve a forgatást.

Össze lehet kavarodni a sok koordinátarendszer között, jobb őket egy jelölésrendszerben felvenni. Erre jó a Denavit-Hartenberg konvenció.

Denavit-Hartenberg szemléltető vidi: [link](https://www.youtube.com/watch?v=nuB_7BkYNMk)

Azért szuper még a D-H, mert a segítségével 4 paraméterrel leírható a csuklók kapcsolata páronként. Ezek:

- $d$, az új KR előzőre vetített x tengelyének távolsága $O$-tól
- $\theta$, a két KR. x tengelyeinek egymásba forgatásához szükséges szög.
- $a$, az origók fe

In [11]:
def DH_d_matrix(d):
    d_matrix=np.eye(4)+np.array([[0,0,0,0],[0,0,0,0],[0,0,0,d],[0,0,0,0]])
    return d_matrix

def DH_theta_matrix(theta):
    #deg_rad átváltás
    theta_rad=theta/360*2*np.pi
    theta_matrix=np.array([[np.cos(theta_rad), -np.sin(theta_rad),0,0],[np.sin(theta_rad), np.cos(theta_rad),0,0],[0,0,1,0],[0,0,0,1]])
    return theta_matrix

def DH_a_matrix(a):
    a_matrix=np.eye(4)+np.array([[0,0,0,a],[0,0,0,0],[0,0,0,0],[0,0,0,0]])
    return a_matrix

def DH_alfa_matrix(alfa):
    #deg_rad átváltás
    alfa_rad=alfa/360*2*np.pi
    alfa_matrix=np.array([[1,0,0,0],[0,np.cos(alfa_rad), -np.sin(alfa_rad),0],[0,np.sin(alfa_rad), np.cos(alfa_rad),0],[0,0,0,1]])
    return alfa_matrix

def DH_matrix(d,theta,a, alfa):
    d_matrix=DH_d_matrix(d)
    theta_matrix=DH_theta_matrix(theta)
    a_matrix=DH_a_matrix(a)
    alfa_matrix=DH_alfa_matrix(alfa)
    
    DH_matrix=np.matmul(theta_matrix, d_matrix)
    DH_matrix=np.matmul(DH_matrix, a_matrix)
    DH_matrix=np.matmul(DH_matrix, alfa_matrix)
    
    return DH_matrix

def DH_alt_matrix(d, theta,a,alfa):
    alfa=alfa/360*2*np.pi
    theta=theta/360*2*np.pi
    dh_matrix=np.array([[np.cos(theta), -np.sin(theta)*np.cos(alfa), np.sin(theta)*np.sin(alfa), a*np.cos(theta)],
                        [np.sin(theta),np.cos(theta)*np.cos(alfa), -np.cos(theta)*np.sin(alfa), a*np.sin(theta)],
                       [0, np.sin(alfa), np.cos(alfa), d],
                       [0,0,0,1]])
    return dh_matrix

[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


In [69]:
from graphics import *
import numpy as np
from functools import reduce


 
win =GraphWin("Bence", 600, 400, autoflush=False)#animáció készítésénél: ld. Zelle pdf-je!
win.setBackground("black")

t=0 #legyen egy változónk az időhöz: a cos, sin függvényebe ezt fgojuk beletenni
radius = 100; #kör sugara
center=Point(250,200) #ez legyen a kp


while win.checkMouse()==None: #azért nem az egyszerűbb megoldást használjuk,mert nem tudjuk, meddig néznénk a kört
    if(t>3600):
        t=0
    
    win.delete("all")#letöröljük az előző képkockát
    centerx=300
    centery=300
    '''szerszámkoord megjelenítése'''
    
    TCP=np.array([[0],[0],[0],[1]])
    dh1=DH_matrix(30,t,100,0)
    dh2=DH_matrix(0,60,50,0)
    dh3=DH_matrix(0,0,0,0)
    
    matrices=[dh1, dh2, dh3]
    points=[Point(centerx,centery)]
    TCP_global=dh1 @ dh2 @ dh3 @ TCP
    Joint_1=dh1 @ dh2 @ TCP
    Joint_2=dh1 @ TCP
    
    coords=[TCP_global, Joint_1, Joint_2]
    
    '''
    TCP_global=dh3 @ dh2 @ dh1 @ TCP
    points.append(to2D(TCP_global.flatten(), centerx, centery))
    '''
    for cord in coords:
        pflat=cord.flatten()
        p2D=to2D(pflat, centerx, centery)
        points.append(p2D)
      
    for p2D in points:
        p2D.setFill("white")
        p2D.draw(win)
     
    colors=["white", "red", "green", "blue"]
    for i in range(1, len(points)):
        line=Line(points[i-1], points[i])
        line.setOutline(colors[i])
        line.draw(win)

    
    centerx=100
    centery=350
    #koordináták rajzolása mellé
    origo3D=[0,0,0]
    origo=to2D(origo3D, centerx,centery)

    pont3D=[0,0,20]
    pont2D=to2D(pont3D, centerx,centery)
    egyenes=Line(origo, pont2D)#így belül is lehet, néha így kényelmesebb
    egyenes.setOutline("white")
    egyenes.draw(win)

    pont3D=[0,20,0]
    pont2D=to2D(pont3D, centerx,centery)
    egyenes=Line(origo, pont2D)#így belül is lehet, néha így kényelmesebb
    egyenes.setOutline("white")
    egyenes.draw(win)

    pont3D=[20,0,0]
    pont2D=to2D(pont3D, centerx,centery)
    egyenes=Line(origo, pont2D)#így belül is lehet, néha így kényelmesebb
    egyenes.setOutline("white")
    egyenes.draw(win)
            
    t+=1
    update(100) #magyarázat a pdf-ben.
     
win.close()